## Analyze appeals data from the Pennsylvania Bureau of Hearings

In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_rows", 200)

In [3]:
df = (
    pd.read_excel(
        "../../data/states/pennsylvania/CAE Data Spreadsheet - by FAA date- (21-RTKL-452).xlsx",
        skiprows = 2,
        dtype = {
            87: float,
            121: float,
            187: float
        }
    )
    .assign(
        total = lambda f: f[21] + f[87] + f[121] + f[187],
    )
    .rename(
        columns = {
            "DISPOSITION CODE": "disp",
            "EXPLANATION OF CODE": "explanation",
            21: "county_3490", # a regular appeal shortly after investigation
            87: "regional_3490",
            121: "county_6341", # a late appeal where new evidence allows for a hearing
            187: "regional_6341"
        }
    )
    .assign(
        county_3490 = lambda f: pd.to_numeric(f["county_3490"], errors="coerce")
    )
    .drop("Unnamed: 0", axis = 1)
)

df.head(3)

,disp,explanation,county_3490,regional_3490,county_6341,regional_6341,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,total
0,AC/AD,ABANDONED IN FAVOR OF DEPARTMENT,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,3.0
1,FA,IN FAVOR OF APPELLANT,638.0,56.0,0.0,0.0,NaN,NaN,NaN,NaN,694.0
2,NaN,NON-PURSUIT (Subcategory of FA total / include...,358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# non pursuits are not mutually-exclusive with these other categories
# except in the second half of 2021
# so calculate them first separately

# cycle through the sheet and split up the sections
start = 0 # initial 2010 selection
end = 11

# collect frames
frames = []
for i, yr in enumerate(range(2010, 2023)): 
    sub = df.iloc[start:end].assign(
        year = yr
    )
    
    start = end + 6
    end = start + 10
    
    # 2022 is just the second half of 2021
    if yr > 2021:
        filtered = (
            sub
            .loc[ lambda x: x["explanation"] == "COUNTY DECIDED NOT TO PURSUE CASE"]
            .assign(
                nps = lambda f: f['county_3490'] + f['regional_3490'],
            )
            [["year", "nps"]]
        )
    else:
        filtered = (
            sub.loc[ lambda x: x["disp"].isna()]
            .rename( columns = {"county_3490": "nps"})
            [["year", "nps"]]
        )
    
    # add to frames
    frames.append(filtered)
    
# non pursuits only
nps = pd.concat(frames).set_index("year")
nps.loc[2021] = nps.loc[2021] + nps.loc[2022]
nps = nps.drop(2022, axis = 0)
nps

,nps
year,
2010,358.0
2011,402.0
2012,253.0
2013,337.0
2014,337.0
2015,303.0
2016,381.0
2017,355.0
2018,318.0


In [5]:
# parse appeals tables minus NPS

# cycle through the sheet and split up the sections
start = 0 # initial 2010 selection
end = 11

# collect frames
frames = []
for i, yr in enumerate(range(2010, 2023)): 
    sub = df.iloc[start:end].copy()
    sub.loc[:,"year"] = yr # add in a year column
    start = end + 6
    end = start + 10
    # add to frames
    
    frames.append(sub.dropna(subset = ["disp"])) # this removes the non-pursuit

# all frames before fix to 2021/2022
adf_temp = pd.concat(frames).dropna(axis = 1)

# need to fix 2021/2022
adf_temp.tail(5)

,disp,explanation,county_3490,regional_3490,county_6341,regional_6341,total,year
198,OA,DISMISSED IN FAVOR OF APPELLANT,0.0,0.0,0.0,0.0,0.0,2022
199,OD,DISMISSED IN FAVOR OF DEPARTMENT,17.0,0.0,1.0,0.0,18.0,2022
200,OS,STIPULATION,0.0,0.0,1.0,0.0,1.0,2022
201,WC/WD,WITHDRAWALS,28.0,1.0,2.0,0.0,31.0,2022
202,XX,ADMINISTRATIVELY CLOSED,0.0,0.0,0.0,0.0,0.0,2022


In [6]:
# create a fixed 2021:
new_2021 = (
    pd.concat(
        [
            adf_temp.loc[ lambda x: x["year"] == 2021 ], 
            adf_temp.loc[ lambda x: x["year"] == 2022 ]
        ]
    )
    .groupby(["disp", "explanation"])
    .pipe( 
        lambda grp: pd.DataFrame(
            {
                "county_3490": grp["county_3490"].sum(),
                "regional_3490": grp["regional_3490"].sum(),
                "county_6341": grp["county_6341"].sum(),
                "regional_6341": grp["regional_6341"].sum(),
                "total": grp["total"].sum(),
                "year": 2021
            }
        )
    )
    .reset_index()
)

# all frames with 2021 added back in
adf = pd.concat([
    adf_temp.loc[ lambda x: x["year"] < 2021],
    new_2021
])

adf.tail(10)

,disp,explanation,county_3490,regional_3490,county_6341,regional_6341,total,year
0,AC/AD,ABANDONED IN FAVOR OF DEPARTMENT,0.0,0.0,1.0,0.0,1.0,2021
1,FA,IN FAVOR OF APPELLANT,287.0,12.0,4.0,0.0,303.0,2021
2,FC/FD,IN FAVOR OF DEPARTMENT,65.0,5.0,1.0,0.0,71.0,2021
3,FI,IN PART DECISION,3.0,0.0,0.0,0.0,3.0,2021
4,NP - NON-PURSUIT,COUNTY DECIDED NOT TO PURSUE CASE,219.0,6.0,0.0,0.0,225.0,2021
5,OA,DISMISSED IN FAVOR OF APPELLANT,0.0,0.0,0.0,0.0,0.0,2021
6,OD,DISMISSED IN FAVOR OF DEPARTMENT,47.0,1.0,1.0,0.0,49.0,2021
7,OS,STIPULATION,0.0,0.0,1.0,0.0,1.0,2021
8,WC/WD,WITHDRAWALS,78.0,3.0,3.0,0.0,84.0,2021
9,XX,ADMINISTRATIVELY CLOSED,0.0,0.0,0.0,0.0,0.0,2021


In [7]:
# 6341 appeals are uncommon, for most calcs we'll remove
(
    adf
    .set_index(["disp", "explanation"])
    .drop(["total", "year"], axis = 1)
    .sum()
    .to_frame("total")
)

,total
county_3490,12385.0
regional_3490,551.0
county_6341,129.0
regional_6341,5.0


In [8]:
# all per year
totals = (
    adf
    .groupby("year")
    .pipe( lambda grp: pd.DataFrame({
        "total": grp["total"].sum(),
    }))
)

totals

,total
year,
2010,1250.0
2011,1194.0
2012,910.0
2013,1141.0
2014,1321.0
2015,1265.0
2016,1201.0
2017,1238.0
2018,1128.0


In [9]:
# all "regular" appeals per year

yearly = (
    adf
    [["year", "county_3490", "regional_3490"]]
    .groupby("year")
    .pipe( lambda grp: pd.DataFrame({
        "total": grp["county_3490"].sum() + grp["regional_3490"].sum(),
    }))
)

yearly

,total
year,
2010,1250.0
2011,1194.0
2012,910.0
2013,1141.0
2014,1321.0
2015,1264.0
2016,1184.0
2017,1196.0
2018,1094.0


In [10]:
# regular hearings (3490) without errors
reg = (
    adf
     # only regular appeals
    .assign(
        total = lambda f: f["county_3490"] + f["regional_3490"]
    )
    [["year", "disp", "explanation", "total"]]
    .loc[ lambda x: x["disp"] != "ADMINISTRATIVELY CLOSED"]
)

reg.head()

,year,disp,explanation,total
0,2010,AC/AD,ABANDONED IN FAVOR OF DEPARTMENT,3.0
1,2010,FA,IN FAVOR OF APPELLANT,694.0
3,2010,FC/FD,IN FAVOR OF DEPARTMENT,380.0
4,2010,FI,IN PART DECISION,4.0
5,2010,OA,DISMISSED IN FAVOR OF APPELLANT,10.0


In [11]:
# all appeals plus non pursued, grouped by year
with_nps = (
    reg
    .groupby(["year"])
    ["total"]
    .sum()
    .to_frame()
    .merge(
        nps,
        on = "year"
    ).assign(
        percent_not_pursued = lambda f: f["nps"]/f["total"]
    )
    .reset_index()
)

with_nps

,year,total,nps,percent_not_pursued
0,2010,1250.0,358.0,0.286400
1,2011,1194.0,402.0,0.336683
2,2012,910.0,253.0,0.278022
3,2013,1141.0,337.0,0.295355
4,2014,1321.0,337.0,0.255110
5,2015,1264.0,303.0,0.239715
6,2016,1184.0,381.0,0.321791
7,2017,1196.0,355.0,0.296823
8,2018,1094.0,318.0,0.290676
9,2019,1009.0,379.0,0.375619


In [12]:
# average hearings per year
(
    with_nps
    .pipe( 
        lambda f: pd.DataFrame(
            {
                "Average": f["total"].mean(),
                "Total": f["total"].sum(),
                "Average not pursued": f["nps"].sum()/f["total"].sum(),
                "Percent average": f["percent_not_pursued"].mean()
            }, index = ["Not pursued plus hearings"]) 
    )
    .T
)

,Not pursued plus hearings
Average,1078.000000
Total,12936.000000
Average not pursued,0.310065
Percent average,0.318130


In [13]:
nps["nps"].sum()

4011.0

In [14]:
# types by percent for 2019
total = reg.loc[ lambda x: x["year"] == 2019]["total"].sum()

(
    reg
    .loc[ lambda x: x["year"] == 2019]
    .groupby(["explanation"])
    .pipe( lambda grp: pd.DataFrame({
        "total": grp.total.sum(),
        "percent": grp.total.sum()/total
    }))
    .sort_values("percent", ascending = False)
)



,total,percent
explanation,,
IN FAVOR OF APPELLANT,660.0,0.654113
IN FAVOR OF DEPARTMENT,176.0,0.174430
DISMISSED IN FAVOR OF DEPARTMENT,109.0,0.108028
WITHDRAWALS,56.0,0.055500
ABANDONED IN FAVOR OF DEPARTMENT,5.0,0.004955
IN PART DECISION,3.0,0.002973
ADMINISTRATIVELY CLOSED,0.0,0.000000
DISMISSED IN FAVOR OF APPELLANT,0.0,0.000000
STIPULATION,0.0,0.000000


In [15]:
# count successes func
def count_success(grp):
    
    total = grp["total"].sum()
    
    success = (
        grp
        .loc[ 
            lambda x: (
                x["disp"]
                .isin([ 
                    "FA", # decide in favor of appellants - includes NPS
                    "OA", # dismissing in favor of appellant
                    "NP - NON-PURSUIT" # counting 2021 non-pursuit
                ])
            )
        ]
        ["total"].sum()
    )
    
    return pd.Series(
        {
            "Total": total,
            "Appellant Success": success,
            "Percent Success": success/total,
        }
    )

In [16]:
# average success, less withdrawals, administrative errors
(
    reg
    .loc[ 
        lambda x: 
        ~x["disp"]
        .isin([
            "XX", # administrative errors
            "WC/WD" # withdrawals
        ])
    ]
    .groupby(['year'])
    .apply(count_success)
    .mean()
    .to_frame("")
)

,
Total,1007.250000
Appellant Success,645.416667
Percent Success,0.648872


In [17]:
# Yearly successes
(
    reg
    .loc[ 
        lambda x: 
        ~x["disp"]
        .isin([
            "XX", # administrative errors
            "WC/WD" # withdrawals
        ])
    ]
    .groupby(['year'])
    .apply( count_success)
)

,Total,Appellant Success,Percent Success
year,,,
2010,1155.0,704.0,0.609524
2011,1130.0,704.0,0.623009
2012,842.0,506.0,0.600950
2013,1058.0,672.0,0.635161
2014,1239.0,694.0,0.560129
2015,1209.0,771.0,0.637717
2016,1118.0,719.0,0.643113
2017,1127.0,730.0,0.647737
2018,1037.0,674.0,0.649952


### Load Pennsylvania Childrens Bureau data counting perpetrators

In [18]:
cb = (
    pd.read_csv("../../data/national/table-5-1.csv")
    .loc[ lambda x: x["State"] == "Pennsylvania"]
    .iloc[0]
    .drop("State")
    .to_frame("perps")
    .reset_index()
    .rename( columns = {"index": "year"} )
    .assign(
        year = lambda f: f["year"].astype(int)
    )
)
    
cb

,year,perps
0,2010,3597.0
1,2011,3295.0
2,2012,3435.0
3,2013,3356.0
4,2014,3279.0
5,2015,4438.0
6,2016,4653.0
7,2017,5062
8,2018,4865
9,2019,4941


In [19]:
# percent appealing each year
yearly_perps = (
    yearly.reset_index()
    .merge(
        cb,
        on = "year",
        how = "left"
    )
    .assign(
        percent_appealing = lambda f: f["total"]/f["perps"]
    )
)

yearly_perps

,year,total,perps,percent_appealing
0,2010,1250.0,3597.0,0.347512
1,2011,1194.0,3295.0,0.362367
2,2012,910.0,3435.0,0.26492
3,2013,1141.0,3356.0,0.339988
4,2014,1321.0,3279.0,0.402867
5,2015,1264.0,4438.0,0.284813
6,2016,1184.0,4653.0,0.254459
7,2017,1196.0,5062,0.23627
8,2018,1094.0,4865,0.224872
9,2019,1009.0,4941,0.20421


In [20]:
# proportion perpetrators appealiing on average
(
    reg.reset_index()
    .loc[ 
        lambda x: x["year"] < 2020 
    ]["total"].sum()
)/ yearly_perps.perps.sum()

0.2539309557273366

---

---

---